In [66]:
import tensorflow as tf
import sys
print(sys.version)
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

3.5.2 |Continuum Analytics, Inc.| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]


In [118]:
movieratings = pd.read_csv('ratings.csv')

users = movieratings.ix[:,0].values
songs = movieratings.ix[:,1].values
ratings = movieratings.ix[:,2].values

#unique users / songs
uni_users = movieratings['userId'].unique()
uni_songs = movieratings['movieId'].unique()

#dict mapping the id to an index
user_map = dict(zip(uni_users,range(len(uni_users))))
song_map = dict(zip(uni_songs,range(len(uni_songs))))

user_idx =  np.array([ user_map[user] for user in users])
song_idx =  np.array([ song_map[song] for song in songs])

print(len(uni_users),len(uni_songs))

perm = np.random.permutation(len(users))
trn_idx = perm[:(len(users)*9)//10]
val_idx = perm[(len(users)*9)//10:]
user_idx_trn, song_idx_trn, ratings_trn = user_idx[trn_idx], song_idx[trn_idx], ratings[trn_idx]
user_idx_val, song_idx_val, ratings_val = user_idx[val_idx], song_idx[val_idx], ratings[val_idx]

def getDfSummary(input_data):
    output_data = input_data.describe(include = 'all').T
    var = pd.DataFrame(data = {'nanvals': pd.Series(), 'number_distinct': pd.Series()})
    for i in range(len(input_data.columns)):
        nanvals = input_data.ix[:,i].isnull().sum()
        number_distinct = len(input_data.ix[:,i].value_counts())
        var = var.append(pd.DataFrame([[nanvals, number_distinct]], columns = ['nanvals', 'number_distinct']))
    var.index = output_data.index.values
    output_data['nanvals'] = var['nanvals']
    output_data['number_distinct'] = var['number_distinct']
    return output_data

output_data = getDfSummary(movieratings)
output_data

671 9066


,count,mean,std,min,25%,50%,75%,max,nanvals,number_distinct
userId,100004.0,3.470113e+02,1.951638e+02,1.0,182.0,3.670000e+02,5.200000e+02,6.710000e+02,0.0,671.0
movieId,100004.0,1.254866e+04,2.636920e+04,1.0,1028.0,2.406500e+03,5.418000e+03,1.639490e+05,0.0,9066.0
rating,100004.0,3.543608e+00,1.058064e+00,0.5,3.0,4.000000e+00,4.000000e+00,5.000000e+00,0.0,10.0
timestamp,100004.0,1.129639e+09,1.916858e+08,789652009.0,965847824.0,1.110422e+09,1.296192e+09,1.476641e+09,0.0,78141.0


In [119]:
def error_usr(yhat, ratings):
    #ratings_mean = np.mean(ratings)
    #ratings_std = np.std(ratings)
    #q_err = np.sum(np.square(yhat - ratings))
    #sorted_idxes = np.argsort(ratings)
    #ratings_ordered, predicted_ordered = ratings[sorted_idxes], yhat[idxes]
    ratings_ordered = np.sort(ratings)[::-1]
    #yhat_ordered = np.sort(yhat)
    predicted_idxes = np.argsort(yhat)
    ratings_by_predict = ratings[predicted_idxes][::-1]
    #print('ratings ordered: ', np.log(ratings_ordered), 'ratings predicted: ', -np.log(ratings_by_predict))
    err = np.sum(np.multiply(np.add(np.log(ratings_ordered), -np.log(ratings_by_predict)), 
                             np.array([1.5**-i for i in range(len(ratings))])))
    return err

class MF_RS():
    def __init__(self, numUsers, numSongs, embedding_dim, reg_lambda = 0.01):
        
        #hyper parameters
        self.batch_size = np.min([1004, numUsers, numSongs]);
        self.numUsers = numUsers
        self.numSongs = numSongs
        self.epochs = 50
        self.reg_lambda = reg_lambda
        
        #embedding matricies for users and songs
        self.userMat = tf.Variable(tf.random_normal([numUsers, embedding_dim]))
        self.songMat = tf.Variable(tf.random_normal([numSongs, embedding_dim]))
        self.userBias = tf.Variable(tf.random_normal([numUsers]))
        self.songBias = tf.Variable(tf.random_normal([numSongs]))
        self.overallBias = tf.Variable(tf.random_normal([1]))
        
        #input tensors for songs, usres, ratings
        self.users = tf.placeholder(tf.int32, shape =(self.batch_size))
        self.songs = tf.placeholder(tf.int32, shape =(self.batch_size))
        self.rating = tf.placeholder(tf.float32, shape = (self.batch_size))
        
        #map each user/song to its feature vector
        self.U = tf.nn.embedding_lookup(self.userMat, self.users)
        self.W = tf.nn.embedding_lookup(self.songMat, self.songs)
        #map each user/song bias to its bias vector
        self.U_bias = tf.nn.embedding_lookup(self.userBias, self.users)
        self.W_bias = tf.nn.embedding_lookup(self.songBias, self.songs)
        
        #predicted rating is dot product of user and song
        bias = self.U_bias+self.W_bias+self.overallBias
        pq = tf.reduce_sum(tf.mul(self.U, self.W), 1)
        self.yhat = pq + bias
        
        self.reg = self.reg_lambda * ( tf.reduce_sum((tf.square(self.U) + tf.square(self.W))) + 
                                 tf.reduce_sum(tf.square(self.U_bias) + tf.square(self.W_bias)))
        self.error = tf.reduce_mean(tf.nn.l2_loss(self.yhat - self.rating))
        self.cost = (self.error + self.reg)/1e4
        self.optimizer = tf.train.AdamOptimizer(learning_rate = .01).minimize(self.cost)
        
        self.session = tf.Session()
        self.session.run(tf.initialize_all_variables())   
        
        
    def train(self, users, songs, ratings, verb = 0):
        
        for i in range(self.epochs):
            
            avg_cost = 0
            perm = np.random.permutation(len(ratings))
            num_batches = len(ratings) // self.batch_size
            
            for b_idx in range(num_batches):
                
                batch = perm[self.batch_size * b_idx:self.batch_size * (b_idx + 1)]
                users_batch = users[batch]
                songs_batch = songs[batch]
                ratings_batch = ratings[batch]
                                
                avg_cost += self.session.run([self.cost, self.optimizer],
                          {self.users:users_batch, self.songs:songs_batch, self.rating:ratings_batch})[0]
                
            if verb > 0:
                print(avg_cost/num_batches)
                
    def test(self, users, songs):
        
        yhat = np.zeros(len(users))
        num_batches = len(users) // self.batch_size
        b_idx = 0
        for b_idx in range(num_batches):
            batch = range(self.batch_size * b_idx,self.batch_size * (b_idx + 1))
            users_batch = users[batch]
            songs_batch = songs[batch]
            yhat[batch] = self.session.run([self.yhat],
                      {self.users:users_batch, self.songs:songs_batch})[0]
        if self.batch_size * (b_idx + 1) < len(users):
            batch = range(self.batch_size * (b_idx + 1), len(users))
            leftover_size = len(batch)
            buffer = self.batch_size - leftover_size
            users_batch = np.concatenate([users[batch], np.array([1]*buffer)])
            songs_batch = np.concatenate([songs[batch], np.array([1]*buffer)])
            y_hat_leftover = self.session.run([self.yhat],
                          {self.users:users_batch, self.songs:songs_batch})[0]
            yhat[batch] = y_hat_leftover[:leftover_size]
        return yhat
    def testdebug(self, users, songs):
        
        yhat = np.zeros(len(users))
        num_batches = len(users) // self.batch_size
        b_idx = -1
        for b_idx in range(num_batches):
            batch = range(self.batch_size * b_idx,self.batch_size * (b_idx + 1))
            users_batch = users[batch]
            songs_batch = songs[batch]
            yhat[batch] = self.session.run([self.yhat],
                      {self.users:users_batch, self.songs:songs_batch})[0]
        if self.batch_size * (b_idx + 1) < len(users):
            batch = range(self.batch_size * (b_idx + 1), len(users))
            leftover_size = len(batch)
            buffer = self.batch_size - leftover_size
            users_batch = np.concatenate([users[batch], np.array([1]*buffer)])
            songs_batch = np.concatenate([songs[batch], np.array([1]*buffer)])
            y_hat_leftover = self.session.run([self.yhat],
                          {self.users:users_batch, self.songs:songs_batch})[0]
            yhat[batch] = y_hat_leftover[:leftover_size]
            #print(yhat)
        #print(yhat)
        return yhat
    def evaluate(self, users, songs, ratings, fcn = 'MSE'):
        if fcn == 'MSE':
            yhat = self.test(users, songs)
            return np.mean(np.square(yhat - ratings))
        if fcn == 'Business':
            num_ratings = len(ratings)
            err_total = 0
            uni_users = np.unique(users)
            for usr in uni_users:
                usr_idxes = users == usr
                usr_idxes = np.where(usr_idxes)
                usr_u = users[usr_idxes]
                sng_u = songs[usr_idxes]
                rtg_u = ratings[usr_idxes]
                yhat = self.testdebug(usr_u, sng_u)
                err_total += error_usr(yhat = yhat, ratings = rtg_u)/num_ratings
            return err_total

In [120]:
run = 'Val'
songmodel = MF_RS(len (uni_users), len(uni_songs), 11, reg_lambda = .01)
print(songmodel.evaluate(user_idx_val, song_idx_val, ratings_val))
songmodel.epochs = 3
songmodel.train(user_idx_trn, song_idx_trn, ratings_trn, verb = 1)
if run == 'Val':
    a = songmodel.evaluate(user_idx_val, song_idx_val, ratings_val, fcn = 'Business')
if run == 'Trn':
    a = songmodel.evaluate(user_idx_trn, song_idx_trn, ratings_trn, fcn = 'MSE')
print(a)

23.5814220485
0.457054281635
0.154780972693
0.0866239292186
0.0313447359127


In [121]:
songmodel = MF_RS(len (uni_users), len(uni_songs), 11, reg_lambda = .01)
songmodel.epochs = 3
for _ in range(10):
    print(songmodel.evaluate(user_idx_val, song_idx_val, ratings_val, fcn = 'Business'))
    songmodel.train(user_idx_trn, song_idx_trn, ratings_trn, verb = 1)

0.034770937168
0.626669133999
0.205100182722
0.108218018744
0.0311419408016
0.0720901806742
0.0543620939464
0.0443470059936
0.028286948051
0.0382320692679
0.0342118327937
0.0314139591725
0.026492935536
0.0293824621546
0.0278241895
0.0266030994452
0.025017616103
0.0256076720891
0.024757828911
0.0240085990126
0.0251152970829
0.0233475896321
0.0227091124198
0.0221250526218
0.0251640453734
0.0215510210273
0.021002736026
0.020452626119
0.0252795736336
0.0199222093754
0.0193972872681
0.0188892046685
0.0255499831389
0.0183951767481
0.017924059238
0.0174743302111
0.0253936792449
0.0170400801754
0.0166423362402
0.0162806781982


In [66]:
c = [True, False, False]
d = np.array([1, 2, 3])
d[c==True]

/home/lee/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel/__main__.py:3: VisibleDeprecationWarning: using a boolean instead of an integer will result in an error in the future
  app.launch_new_instance()


1

In [36]:
a = np.array([1, 2, 3, 4, 5])
b = np.array([1, 2, 3, 4, 5])
c = np.array([4, 3, 2, 5, 1])
#unique users / songs
uni_a = np.unique(a)
uni_b = np.unique(b)

#dict mapping the id to an index
a_map = dict(zip(uni_a,range(len(uni_a))))
b_map = dict(zip(uni_b,range(len(uni_b))))

user_idx =  np.array([ a_map[user] for user in a])
song_idx =  np.array([ b_map[song] for song in b])
model = MF_RS(len(uni_a), len(uni_b), 7)
np.random.seed(2)
model.train(user_idx, song_idx, c)

KeyboardInterrupt: 

In [16]:
edims = [10, 30, 50]
lambdas = [10**i for i in range(-4, -1)]
errmat = np.zeros([len(edims), len(lambdas)])
for eidx, edim in enumerate(edims):
    for lidx, lamb in enumerate(lambdas):
        songmodel = MF_RS(len (uni_users), len(uni_songs), edim, reg_lambda=lamb)
        print("accuracy before training", songmodel.evaluate(user_idx_val, song_idx_val, ratings_val))
        np.random.seed(1)
        songmodel.train(user_idx_trn, song_idx_trn, ratings_trn)
        err = songmodel.evaluate(user_idx_val, song_idx_val, ratings_val)
        print("MSE after training with edim: ", edim, " and lambda: ", lamb, ": ", err)
        errmat[eidx, lidx] = err
errmat

accuracy before training 22.8770905418
MSE after training with edim:  10  and lambda:  0.0001 :  1.69646169488
accuracy before training 29.9246706894
MSE after training with edim:  10  and lambda:  0.001 :  1.65306456763
accuracy before training 19.0567726014
MSE after training with edim:  10  and lambda:  0.01 :  1.22921916365
accuracy before training 45.5816602463


KeyboardInterrupt: 